In [ ]:
from aiida import load_profile 
profile =  load_profile()

In [ ]:
from aiida.orm import Str, Dict, Bool, load_node, Float
from aiida.engine import calcfunction
from akaikkr_testscript import get_kkr_struc_from_cif, change_atomic_type
from pyakaikkr import AkaikkrJob, make_displc_list
import os
from aiida.engine import run, submit
from tools.aiida_support import wait_for_node_finished
from aiida.plugins import DataFactory
from aiida.orm import load_code
import matplotlib.pyplot as plt
import pandas as pd

ArrayData = DataFactory('array')


In [ ]:
# parameters
SPECX =  Str("/home/max/Documents/akaikkr/AkaiKKRprogram.2022.0721.ifort/akaikkr_cnd/specx")
CIFFILEPATH = Str("structure/Fe-Im3m.cif")
DISPLC = Bool(True)

In [ ]:
@calcfunction
def _structure_param(
        akaikkr_exe: Dict, displc: Bool, ciffilepath: Str
        ) -> dict:
    """define Cu common kkr parameters

    internal options:
        use_bravais (bool, optional): use bravias in the akaikkr format. Defaults to True.
        remove_temperaryfiles (bool, optional): remove temporary files after executing geometry by specx. Defaults to False.
        directory (str, optional): directory to place input and output files in getting geometry. Defaults to "temporary".


    Args:
        akaikkr_exe (Dict): specx and fmg
        displc (Bool): include displc or not. True if akaikkr_cnd.
        ciffilepath (Str, optional): cif file name. Defaults to "../structure/Cu-Fm3m.cif".

    Returns:
        dict: kkr input parameters
    """
    use_bravais=True
    remove_temperaryfiles=True,
    directory="temporary"
    
    ciffilepath = ciffilepath.value
    akaikkr_exe = akaikkr_exe.get_dict()
    displc = displc.value
    param = get_kkr_struc_from_cif(
        ciffilepath=ciffilepath, akaikkr_exe=akaikkr_exe["specx"], displc=displc,
        use_bravais=use_bravais, remove_temperaryfiles=remove_temperaryfiles,
        directory=directory)
    
    # param["a"] = 6.82

    # to compare the results with reference
    # param = change_atomic_type(param, {"Cu_4a_0": "Cu"})
    

    if displc:
        param["displc"] = make_displc_list(param["anclr"])    

    return Dict(dict=param)

akaikkr_exe = {}
akaikkr_exe["specx"] = SPECX
akaikkr_exe = Dict(dict=akaikkr_exe)
kkr_structure_param = _structure_param(akaikkr_exe, DISPLC, CIFFILEPATH)
kkr_structure_param.get_dict()

In [ ]:
magtype = Str("mag")

# Go

In [ ]:
@calcfunction
def _Fe_go_params(record:Str):
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = record
    return Dict(dict=kkr_param)

kkr_param = _Fe_go_params(Str("init"))

In [ ]:
code = load_code('akaikkr_go_calcjob_localhost')
code

In [ ]:
builder = code.get_builder()
builder.go = Str("go")
builder.magtype = magtype
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
gofuture = submit(builder)
print(gofuture)
wait_for_node_finished(gofuture)

In [ ]:
gofuture.outputs.structure.get_ase()

In [ ]:
gofuture.outputs.retrieved.list_object_names()

In [ ]:
gofuture.outputs.remote_folder.listdir()

In [ ]:
gofuture.outputs.structure

In [ ]:
potential = gofuture.outputs.potential
potential

# 2nd Go with remote potential

In [ ]:
kkr_param = _Fe_go_params(Str("2nd"))

In [ ]:
remotepotential = gofuture.outputs.remote_folder

In [ ]:
remotepotential.computer

In [ ]:
builder = code.get_builder()
builder.go = Str("go")
builder.magtype = magtype
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
builder.potential = remotepotential
gofuture2nd = submit(builder)
print(gofuture2nd)
wait_for_node_finished(gofuture2nd)

In [ ]:
gofuture2nd.outputs.results.get_dict()

# 2nd Go with local potential

In [ ]:
kkr_param = _Fe_go_params(Str("2nd"))
potential = gofuture.outputs.potential
potential

In [ ]:
builder = code.get_builder()
builder.go = Str("go")
builder.magtype = magtype
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
builder.potential = potential
gofuture2nd = submit(builder)
print(gofuture2nd)
wait_for_node_finished(gofuture2nd)

In [ ]:
gofuture2nd.outputs.results.get_dict()

In [ ]:
gofuture2nd.pk

In [ ]:
!verdi node graph generate $gofuture2nd.pk

In [ ]:
from wand.image import Image as WImage
img = WImage(filename=f'{gofuture2nd.pk}.dot.pdf')
img

# Dos

In [ ]:
@calcfunction
def _Cu_dos_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "2nd"
    kkr_param["ewidth"] = 2.0
    return Dict(dict=kkr_param)

def _Fe_dos_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "2nd"
    kkr_param["ewidth"] = 2.0
    return Dict(dict=kkr_param)

kkr_param = _Fe_dos_params()

In [ ]:
code = load_code('akaikkr_dos_calcjob_localhost')

builder = code.get_builder()
builder.go = Str("dos")
builder.magtype = magtype
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
builder.potential = potential
dosfuture = submit(builder)
print(dosfuture)
wait_for_node_finished(dosfuture)

In [ ]:
dosfuture.outputs.results.get_dict()['type_of_site']

In [ ]:
dos = dosfuture.outputs.dos
dos.attributes

In [ ]:
def dos_to_arrays(dosdata: ArrayData):
    energy = dosdata.get_array('energy')
    dos = dosdata.get_array('dos')    
    return energy, dos

energy, dos = dos_to_arrays(dosfuture.outputs.dos)
plt.plot(energy, dos[0])
plt.plot(energy, dos[1])

In [ ]:
def pdos_to_arrays(pdos: ArrayData):
    pdos_energy = pdos.get_array('energy')
    pdos_value = pdos.get_array('pdos')
    return pdos_energy, pdos_value

pdos_energy, pdos_value = pdos_to_arrays(dosfuture.outputs.pdos)
for i in range(pdos_value.shape[3]):
    plt.plot(pdos_energy, pdos_value[0,0,:,i])


# spc

In [ ]:
structure_data = gofuture2nd.outputs.structure

In [ ]:
@calcfunction
def _Cu_spc_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "2nd"
    kkr_param["magtyp"] = "nmag"
    kkr_param["ewidth"] = 2.0
    return Dict(dict=kkr_param)

def _Fe_spc_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "2nd"
    kkr_param["magtyp"] = "mag"
    kkr_param["ewidth"] = 2.0
    return Dict(dict=kkr_param)

kkr_param = _Fe_spc_params()

In [ ]:
code = load_code('akaikkr_spc_calcjob_localhost')

builder = code.get_builder()
builder.magtype = magtype
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
builder.potential = potential
builder.structure_data = structure_data

spcfuture = submit(builder)
print(spcfuture)
wait_for_node_finished(spcfuture)

# A(w,k) plot

In [ ]:
Awk_up = spcfuture.outputs.Awk_up
Awk_dn = spcfuture.outputs.Awk_dn

klabeldict = spcfuture.outputs.klabel.get_dict()

In [ ]:
klabel = []
for vpath in klabeldict["kpath"]:
    for v in vpath:
        k = str(list(v.keys())[0])
        k = "$"+k+r"$"
        klabel.append(k)
klabel

In [ ]:
with Awk_up.open(Awk_up.filename) as handle:
    from pyakaikkr import AwkReader
    awk_up = AwkReader(handle)
with Awk_dn.open(Awk_dn.filename) as handle:
    from pyakaikkr import AwkReader
    awk_dn = AwkReader(handle)

In [ ]:
from pyakaikkr import AwkPlotter
import matplotlib.pyplot as plt
%matplotlib inline
awkplotter = AwkPlotter(".")
fig, axes = plt.subplots(1,2)
for awk,ax in zip([awk_up, awk_dn], axes):
    awkplotter.make(awk.kcrt, awk.Awk, awk.kdist, awk.energy, klabel, ax=ax)
fig.show()

In [ ]:
!verdi node graph generate $spcfuture.pk
from wand.image import Image as WImage
img = WImage(filename=f'{spcfuture.pk}.dot.pdf')
img

# Jij

In [ ]:
future = gofuture2nd
potential = future.outputs.potential
structure_data = future.outputs.structure


In [ ]:
@calcfunction
def _Fe_jij_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "2nd"
    kkr_param["ewidth"] = 2.0
    return Dict(dict=kkr_param)

kkr_param = _Fe_jij_params()

In [ ]:
code = load_code('akaikkr_jij_calcjob_localhost')

builder = code.get_builder()
builder.magtype = magtype

builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
builder.potential = potential

spcfuture = submit(builder)
print(spcfuture)
wait_for_node_finished(spcfuture)

In [ ]:
spcfuture.outputs.Tc

In [ ]:
def jij_to_dataframe(jijdata):
    df = pd.DataFrame()
    for key,value in jijdata.items():
        df[key] = value
    return df

jijdata = spcfuture.outputs.Jij.get_dict()
df = jij_to_dataframe(jijdata)
display(df)
df['distance'] = df['distance'].values.astype(float)
df['J_ij(meV)'] = df['J_ij(meV)'].values.astype(float)
df.plot(x='distance', y='J_ij(meV)')

# Tc

In [ ]:
future = gofuture2nd
potential = future.outputs.potential
structure_data = future.outputs.structure


In [ ]:
code = load_code('akaikkr_tc_calcjob_localhost')

builder = code.get_builder()
builder.magtype = magtype
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
builder.potential = potential

tcfuture = submit(builder)
print(tcfuture)
wait_for_node_finished(tcfuture)

In [ ]:
tcfuture.outputs.Tc

# fspin


In [ ]:
@calcfunction
def _Fe_fsm_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "init"
    kkr_param["magtyp"] = "mag"
    kkr_param["ewidth"] = 2.0
    kkr_param["pmix"] = "0.02ch"
    return Dict(dict=kkr_param)

kkr_param = _Fe_fsm_params()
kkr_param

In [ ]:
code = load_code('akaikkr_fsm_calcjob_localhost')
import numpy as np

true = Bool(True)
true.store()
results = []
for fspin in np.linspace(0.0, 3.0, 10):
    kkr_param = kkr_param.get_dict()
    if fspin==0.0:
        kkr_param["record"] = "init"
        potential = Str("")
    else:
        kkr_param["record"] = "2nd"
    kkr_param = Dict(dict=kkr_param)
    builder = code.get_builder()
    builder.magtype = magtype
    builder.displc = DISPLC
    builder.structure = kkr_structure_param
    builder.parameters = kkr_param
    builder.fspin = Float(fspin)
    builder.retrieve_potential = true
    print("potential", potential)
    builder.potential = potential

    spcfuture = submit(builder)
    print(spcfuture)
    wait_for_node_finished(spcfuture, 3)
    
    m = spcfuture.outputs.results.get_dict()['total_moment']
    e = spcfuture.outputs.results.get_dict()['total_energy']
    c = spcfuture.outputs.results.get_dict()['convervence']
    print([m,e,c])
    results.append([m,e,c])
    
    potential = spcfuture.outputs.potential

In [ ]:
df = pd.DataFrame(results, columns=["fspin","total_energy","convergence"])
df.plot(x="fspin", y="total_energy")

# lmd

In [ ]:
import numpy as np
def _Fe_lmd_structure_param(akaikkr_exe: Dict, displc: Bool, ciffilepath: Str):

    use_bravais=True
    remove_temperaryfiles=True
    directory="temporary"
    
    ciffilepath = ciffilepath.value
    akaikkr_exe = akaikkr_exe.get_dict()
    displc = displc.value
    
    param = get_kkr_struc_from_cif(
        ciffilepath=ciffilepath, akaikkr_exe=akaikkr_exe["specx"], displc=displc,
        use_bravais=use_bravais, remove_temperaryfiles=remove_temperaryfiles,
        directory=directory)

    # param["a"] = 5.27
    param["magtyp"] = "lmd"
    param["rmt"] = [1.0 for i in range(param["ntyp"])]

    # lmd must increase ncomp
    ncmp = param["ncmp"][0]
    param["ncmp"] = [ncmp*2]

    # alloy isn't allowed now.
    # extend sizes twice
    anclr_list = []
    for anclr in param["anclr"]:
        _x = anclr
        if len(_x) != 1:
            raise ValueError("Each anclr must be 1.")
        _x.extend(_x)
        anclr_list.append(_x)
    param["anclr"] = anclr_list

    conc_list = []
    for conc in param["conc"]:
        conc = np.array(conc)
        _x = conc*0.5
        _x = _x.tolist()
        if len(_x) != 1:
            raise ValueError("Each conc must be 1.")
        _x.extend(_x)
        conc_list.append(_x)
    param["conc"] = conc_list

    if displc:
        param["displc"] = make_displc_list(param["anclr"])    
        
    return Dict(dict=param)

akaikkr_exe = {}
akaikkr_exe["specx"] = SPECX.value
akaikkr_exe = Dict(dict=akaikkr_exe)
kkr_structure_param = _Fe_lmd_structure_param(akaikkr_exe, DISPLC, CIFFILEPATH)
kkr_structure_param.get_dict()


In [ ]:
@calcfunction
def _Fe_lmd_params():
    directory = "dummy"
    job = AkaikkrJob(directory)
    kkr_param = job.default
    kkr_param["record"] = "init"
    kkr_param["ewidth"] = 2.0
    return Dict(dict=kkr_param)

kkr_param = _Fe_lmd_params()

In [ ]:
code = load_code('akaikkr_go_calcjob_localhost')

builder = code.get_builder()
builder.go = Str("go")
builder.magtype = Str('lmd')
builder.displc = DISPLC
builder.structure = kkr_structure_param
builder.parameters = kkr_param
gofuture = submit(builder)
print(gofuture)
wait_for_node_finished(gofuture)

In [ ]:
gofuture.outputs.potential